# Secondary Structure Shifted Word2Vec Encoder

This demo creates a dataset of sequence segments derived from a non-redundent set. The dataset contains the seuqence segment, the DSSP Q8 and DSSP Q3 code of the center residue in a sequnece segment, and a 3-gram shifted Word2Vec encoding of the seuqnece segment.

## Imports

In [12]:
from pyspark import SparkConf, SparkContext, SQLContext
from mmtfPyspark.ml import proteinSequenceEncoder
from mmtfPyspark.mappers import structureToPolymerChains
from mmtfPyspark.filters import containsLProteinChain
from mmtfPyspark.datasets import secondaryStructureSegmentExtractor
from mmtfPyspark.webfilters import pisces
from mmtfPyspark.io import MmtfReader
import time

## Configure Spark Context

In [13]:
conf = SparkConf() \
        .setMaster("local[*]") \
        .setAppName("secondaryStructureShiftedWord2VecEncodeDemo")
sc = SparkContext(conf = conf)

## Read in, filter and sample Hadoop Sequence Files

In [14]:
path = "../../resources/mmtf_reduced_sample/"

sequenceIdentity = 20
resolution = 2.0
fraction = 0.1
seed = 123

pdb = MmtfReader \
        .readSequenceFile(path, sc) \
        .flatMap(structureToPolymerChains()) \
        .filter(pisces(sequenceIdentity, resolution)) \
        .filter(containsLProteinChain()) \
        .sample(False, fraction, seed)

## Extract Secondary Structure Segments

In [15]:
segmentLength = 25
data = secondaryStructureSegmentExtractor.getDataset(pdb, segmentLength).cache()

## Add Word2Vec encoded feature vector

In [16]:
encoder = proteinSequenceEncoder(data)

windowSize = (segmentLength -1) // 2
vectorSize = 50
data = encoder.shifted3GramWord2VecEncode(windowSize, vectorSize).cache()

root
 |-- ngram0: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- structureChainId: string (nullable = false)
 |-- sequence: string (nullable = false)
 |-- labelQ8: string (nullable = false)
 |-- labelQ3: string (nullable = false)
 |-- feature2: vector (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature0: vector (nullable = true)
 |-- features: vector (nullable = true)



## Show dataset schema and few rows of data

In [17]:
data.printSchema()
data.show(10, False)

root
 |-- structureChainId: string (nullable = false)
 |-- sequence: string (nullable = false)
 |-- labelQ8: string (nullable = false)
 |-- labelQ3: string (nullable = false)
 |-- ngram0: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- feature0: vector (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature2: vector (nullable = true)
 |-- features: vector (nullable = true)

+----------------+-------------------------+-------+-------+----------------------------------------+----------------------------------------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
df = data.toPandas()

df.head(10)

,structureChainId,sequence,labelQ8,labelQ3,ngram0,ngram1,ngram2,feature0,feature1,feature2,features
0,1B8D.K,ACFAKYSYLKNAGEAGDSPEKINKC,T,C,"[ACF, AKY, SYL, KNA, GEA, GDS, PEK, INK]","[CFA, KYS, YLK, NAG, EAG, DSP, EKI, NKC]","[FAK, YSY, LKN, AGE, AGD, SPE, KIN, KC]","[-0.9183551892638206, 0.10246020182967186, 0.0...","[-1.489936202764511, -0.740118620917201, -0.83...","[-2.513897404074669, 0.06753388233482838, -2.4...","[-1.6407295987010002, -0.19004151225090027, -1..."
1,1AOL.A,CNTAWNRLKLDQVTHKSSEGFYVCP,H,H,"[CNT, AWN, RLK, LDQ, VTH, KSS, EGF, YVC]","[NTA, WNR, LKL, DQV, THK, SSE, GFY, VCP]","[TAW, NRL, KLD, QVT, HKS, SEG, FYV, CP]","[-2.5930849760770798, -2.0326022058725357, -0....","[0.41302028857171535, 0.3981254193931818, -2.8...","[0.537737213075161, 0.66351268440485, 1.339030...","[-0.5474424914767345, -0.3236547006915013, -0...."
2,1FO8.A,DPSLWCVSAWNDNGKEQMVDSSKPE,T,C,"[DPS, LWC, VSA, WND, NGK, EQM, VDS, SKP]","[PSL, WCV, SAW, NDN, GKE, QMV, DSS, KPE]","[SLW, CVS, AWN, DNG, KEQ, MVD, SSK, PE]","[0.39397716522216797, -0.7632312038913369, -2....","[-2.0650729089975357, 0.6409277943894267, 1.47...","[-0.3773380406200886, -0.6675133854150772, -1....","[-0.6828112614651521, -0.26327226497232914, -0..."
3,1FSG.C,GCCYDFNEMFRDFDHVAVLSDAARK,C,C,"[GCC, YDF, NEM, FRD, FDH, VAV, LSD, AAR]","[CCY, DFN, EMF, RDF, DHV, AVL, SDA, ARK]","[CYD, FNE, MFR, DFD, HVA, VLS, DAA, RK]","[1.0625259149819613, -0.14590864814817905, -1....","[0.26220549270510674, 0.2825535926967859, 0.57...","[-0.19324837252497673, -0.4644614190328866, -0...","[0.3771610117206971, -0.10927215816142659, -0...."
4,1FSG.C,IDKILLPGGLVKDRVEKLAYDIHRT,H,H,"[IDK, ILL, PGG, LVK, DRV, EKL, AYD, IHR]","[DKI, LLP, GGL, VKD, RVE, KLA, YDI, HRT]","[KIL, LPG, GLV, KDR, VEK, LAY, DIH, RT]","[0.4818373806774616, 0.9853988215327263, 1.190...","[-0.31511412002146244, 0.7385272681713104, -1....","[0.2123634796589613, -1.8934636302292347, -1.1...","[0.1263622467716535, -0.05651251350839933, -0...."
5,1DWK.E,KAKKDLSFAEIADGTGLAEAFVTAA,T,C,"[KAK, KDL, SFA, EIA, DGT, GLA, EAF, VTA]","[AKK, DLS, FAE, IAD, GTG, LAE, AFV, TAA]","[KKD, LSF, AEI, ADG, TGL, AEA, FVT, AA]","[2.6306391060352325, 0.03961627185344696, 0.42...","[0.0343342712149024, -0.8195096179842949, -0.9...","[1.6798848807811737, 1.3165665939450264, -0.14...","[1.4482860860104363, 0.17889108260472616, -0.2..."
6,1C1K.A,KAYRKILNIDSQKAKNVFIETVKSC,H,H,"[KAY, RKI, LNI, DSQ, KAK, NVF, IET, VKS]","[AYR, KIL, NID, SQK, AKN, VFI, ETV, KSC]","[YRK, ILN, IDS, QKA, KNV, FIE, TVK, SC]","[1.8925879001617432, 1.804948329925537, 0.5760...","[1.2777476608753204, -0.03619726584292948, 0.9...","[0.5978518202900887, 0.5813960200175643, 1.985...","[1.256062460442384, 0.783382361366724, 1.17874..."
7,1A9X.F,LSSYLKRHNIVAIADIDTRKLTRLL,E,E,"[LSS, YLK, RHN, IVA, IAD, IDT, RKL, TRL]","[SSY, LKR, HNI, VAI, ADI, DTR, KLT, RLL]","[SYL, KRH, NIV, AIA, DID, TRK, LTR, LL]","[-0.7543083010241389, -1.542003259062767, -3.2...","[-1.0870714522898197, -1.1201349152252078, 0.4...","[-0.7313865143805742, 0.42452575266361237, -0....","[-0.8575887558981776, -0.7458708072081208, -1...."
8,1B8D.K,SAYIAAFTFTRDRLCVPRDMSSQAG,H,H,"[SAY, IAA, FTF, TRD, RLC, VPR, DMS, SQA]","[AYI, AAF, TFT, RDR, LCV, PRD, MSS, QAG]","[YIA, AFT, FTR, DRL, CVP, RDM, SSQ, AG]","[0.6931012012064457, -0.8556204698979855, -2.4...","[0.5563492868095636, -0.08643705036956817, -3....","[-0.2679368369281292, -0.07683034287765622, 1....","[0.32717121702929336, -0.33962928771506995, -1..."
9,1A9X.F,TTAEAYSWTQGSWTLTGGLPQAKKE,C,C,"[TTA, EAY, SWT, QGS, WTL, TGG, LPQ, AKK]","[TAE, AYS, WTQ, GSW, TLT, GGL, PQA, KKE]","[AEA, YSW, TQG, SWT, LTG, GLP, QAK, KE]","[2.156272128224373, 1.9097231291234493, -1.068...","[-2.4091091454029083, 1.0913497265428305, 1.70...","[1.7862593904137611, 2.5997653156518936, -1.62...","[0.5111407910784086, 1.8669460571060579, -0.32..."


## Terminate Spark Context

In [8]:
sc.stop()